In [2]:
import os
import cv2
import numpy as np
import random
from keras.models import load_model

In [4]:
import os
import cv2
import numpy as np
def load_dataset(image_dir, density_dir, input_size=(768, 1024), output_size=(192, 256)):
    images = []
    densities = []
    i = 0
    image_files = os.listdir(image_dir)
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        density_path = os.path.join(density_dir, image_file.replace('.jpg', '.npy'))
        
        # Load and preprocess image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, input_size)
        image = image / 255.0
        
        # Load and preprocess density map
        density = np.load(density_path)
        old_h, old_w = density.shape
        density = cv2.resize(density, output_size)

        # Rescale density to preserve total count
        scaling_factor = (old_h * old_w) / (output_size[0] * output_size[1])
        density = density * scaling_factor

        density = density[..., np.newaxis]

        images.append(image)
        densities.append(density)

    return np.array(images), np.array(densities)



# Provide your directories
image_dir= 'crowd_wala_dataset/train_data/images'
density_dir = 'density_maps/train_set/train_data_sigma_9/'
# Load full dataset into memory
X, Y = load_dataset(image_dir, density_dir)
print(X.shape, Y.shape)

(400, 1024, 768, 3) (400, 256, 192, 1)


In [6]:
# Split into train and validation
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

print(f"Train shapes: {X_train.shape}, {Y_train.shape}")
print(f"Validation shapes: {X_val.shape}, {Y_val.shape}")

Train shapes: (360, 1024, 768, 3), (360, 256, 192, 1)
Validation shapes: (40, 1024, 768, 3), (40, 256, 192, 1)


In [3]:
for i in Y_train:
    print(np.sum(i))

218.1647
216.49251
204.52306
81.704414
25.231754
150.39014
29.986843
128.2212
128.78969
116.53313
207.87088
83.893265
136.19829
29.672665
144.23419
86.61216
104.33358
82.69711
69.71112
166.8442
266.00052
57.620567
299.8041
96.57166
26.173079
40.040688
42.728138
148.54416
236.33632
26.362684
104.09512
51.181454
40.46443
140.16698
113.19511
47.391148
212.86404
113.3421
243.89394
147.17593
41.833145
172.65565
227.73302
101.90081
55.517296
49.21595
161.56868
149.0795
145.30496
29.589256
24.104342
176.76689
90.1107
289.51712
281.09967
110.25662
232.48566
97.231544
157.94691
37.013268
26.089153
411.05252
142.2281
85.6056
259.64682
11.064617
134.7923
40.637623
147.20763
47.5861
297.18546
164.20546
120.32141
26.558329
98.55382
195.72226
235.77989
104.39292
46.48849
149.68816
14.504221
135.89484
41.44818
172.46696
362.70834
264.1342
91.945694
57.486546
41.843437
176.85207
63.933533
105.587845
100.5764
115.39583
79.01911
128.81725
196.22646
122.88568
338.15067
89.44609
41.944016
137.28067
548.88

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, initializers
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam

class CBAMLayer(tf.keras.layers.Layer):
    def __init__(self, reduction_ratio=8, **kwargs):
        super(CBAMLayer, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        self.channel = input_shape[-1]
        self.shared_dense_one = tf.keras.layers.Dense(self.channel // self.reduction_ratio,
                                                      activation='relu')
        self.shared_dense_two = tf.keras.layers.Dense(self.channel)

        self.conv_spatial = tf.keras.layers.Conv2D(filters=1,
                                                   kernel_size=7,
                                                   padding='same',
                                                   activation='sigmoid')

    def call(self, inputs):
        # ----- Channel Attention -----
        avg_pool = tf.reduce_mean(inputs, axis=[1, 2], keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=[1, 2], keepdims=True)

        avg_out = self.shared_dense_two(self.shared_dense_one(avg_pool))
        max_out = self.shared_dense_two(self.shared_dense_one(max_pool))
        scale = tf.nn.sigmoid(avg_out + max_out)
        x = inputs * scale

        # ----- Spatial Attention -----
        avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)
        scale = self.conv_spatial(concat)
        x = x * scale
        return x

Model_Input=Input(shape=(1024,768,3))
from keras.applications.vgg16 import VGG16

base_model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(1024, 768,3))

conv_base = Model(inputs=base_model.input, outputs=base_model.get_layer('block3_conv3').output)

conv_base.summary()

x = conv_base(Model_Input)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1024, 768, 3)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 1024, 768, 64)  │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 1024, 768, 64)  │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 512, 384, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 512, 384, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 512, 384, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 256, 192, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 256, 192, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 256, 192, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 256, 192, 256)  │       590,080 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,735,488 (6.62 MB)

 Trainable params: 1,735,488 (6.62 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
a1=Conv2D(512,3,padding='same',activation='relu')(x)
a2=Conv2D(256,3,padding='same',activation='relu')(a1)
a3=Conv2D(128,3,padding='same',activation='relu')(a2)
a4=Conv2D(64,3,padding='same',activation='relu')(a3)

b1=Conv2D(512,3,padding='same',activation='relu',dilation_rate=2)(x)
b2=Conv2D(256,3,padding='same',activation='relu',dilation_rate=2)(b1)
b3=Conv2D(128,3,padding='same',activation='relu',dilation_rate=2)(b2)
b4=Conv2D(64,3,padding='same',activation='relu',dilation_rate=2)(b3)

c1=Conv2D(512,3,padding='same',activation='relu',dilation_rate=3)(x)
c2=Conv2D(256,3,padding='same',activation='relu',dilation_rate=3)(c1)
c3=Conv2D(128,3,padding='same',activation='relu',dilation_rate=3)(c2)
c4=Conv2D(64,3,padding='same',activation='relu',dilation_rate=3)(c3)

merged_ouput=concatenate([a4,b4,c4])
d1=CBAMLayer()(merged_ouput)
e1=Conv2D(1,1)(d1)

model=Model(inputs=Model_Input,outputs=e1)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Compile model
model.compile(optimizer=Adam(1e-5), loss='mse')

# Early stopping callback
early_stopping = EarlyStopping(
    patience=5,         # Stop after 5 epochs without improvement
    verbose=1,
)

# Fit model with validation data
model.fit(
    X_train, Y_train,
    batch_size=1,
    epochs=100,
    validation_data=(X_val, Y_val),
    callbacks=[early_stopping]
)

Epoch 1/100


In [ ]:
model.save("crowd_counting_model2.keras")
# from tensorflow.keras.models import load_model
# model = load_model("crowd_counting_model.keras")

c:\Users\Shivaay Dhondiyal\Desktop\shivaay\coding\2_development\projects\10_DL_crowd_counting_analysis\venv\lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 28 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [39]:
import os
import cv2
import numpy as np

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image


test_image_dir = 'crowd_wala_dataset/test_data/images/'
predicted_density_dir = 'density_maps/predicted/'
os.makedirs(predicted_density_dir, exist_ok=True)

for image_file in os.listdir(test_image_dir):
    image_path = os.path.join(test_image_dir, image_file)
    
    # Load and preprocess image (adjust this to your model's input needs)
    image = preprocess_image(image_path)
    pred_density = model.predict(image)[0, :, :, 0]  # assuming output shape [1, h, w, 1]

    # Save predicted density map as .npy
    pred_filename = image_file.replace('.jpg', '.npy')
    np.save(os.path.join(predicted_density_dir, pred_filename), pred_density)

    print(f"Predicted and saved density map for {image_file}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step
Predicted and saved density map for IMG_1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
Predicted and saved density map for IMG_10.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Predicted and saved density map for IMG_100.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Predicted and saved density map for IMG_101.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Predicted and saved density map for IMG_102.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
Predicted and saved density map for IMG_103.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Predicted and saved density map for IMG_104.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Predicted and saved density map for IMG_105.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
Predicted and saved density map for IMG_106.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Predicted and saved density map for IMG_107.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Predicted and saved density map for IMG_108.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Predict

In [40]:
import os
import scipy.io
import cv2
import numpy as np
import matplotlib.pyplot as plt

def load_ground_truth(mat_file):
    data = scipy.io.loadmat(mat_file)
    points = data['image_info'][0,0][0,0][0]
    return points

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Set paths
test_image_dir = 'crowd_wala_dataset/test_data/images/'
test_gt_dir = 'crowd_wala_dataset/test_data/ground_truth/'
test_files = os.listdir(test_image_dir)

In [36]:
# Start evaluation
total_absolute_error = 0
total_samples = 0

for file in test_files:
    image_path = os.path.join(test_image_dir, file)
    gt_path = os.path.join(test_gt_dir, "GT_" + file.replace(".jpg", ".mat"))
    
    # Load and preprocess
    test_image = preprocess_image(image_path)
    
    # Predict
    predicted_density = model.predict(test_image)
    predicted_density = np.squeeze(predicted_density)
    predicted_count = predicted_density.sum()
    
    # Ground truth count
    points = load_ground_truth(gt_path)
    gt_count = len(points)
    
    # Compare
    error = abs(predicted_count - gt_count)
    total_absolute_error += error
    total_samples += 1
    
    print(f"Image: {file}")
    print(f"Predicted Count: {predicted_count:.2f}")
    print(f"Ground Truth Count: {gt_count}")
    print(f"Absolute Error: {error:.2f}\n")

# Compute MAE (Mean Absolute Error)
mae = total_absolute_error / total_samples
print(f"Mean Absolute Error on test set: {mae:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
Image: IMG_1.jpg
Predicted Count: 48.42
Ground Truth Count: 23
Absolute Error: 25.42

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Image: IMG_10.jpg
Predicted Count: 98.50
Ground Truth Count: 181
Absolute Error: 82.50

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Image: IMG_100.jpg
Predicted Count: 139.45
Ground Truth Count: 158
Absolute Error: 18.55

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Image: IMG_101.jpg
Predicted Count: 0.94
Ground Truth Count: 37
Absolute Error: 36.06

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Image: IMG_102.jpg
Predicted Count: 75.82
Ground Truth Count: 70
Absolute Error: 5.82

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Image: IMG_103.jpg
Predicted Count: 153.30
Ground Truth Count: 57
Absolute Error: 96.30

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
Image: IMG_104.jpg
Predicted Count: 223.02
Ground Truth Count: 44
Absolute Error: 179.02

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Image: IMG_105.jpg
Predicted Count: 136.94
Ground Truth Count: 228
Abso